# Day 15
https://adventofcode.com/2018/day/15

In [1]:
import aocd
data = aocd.get_data(year=2018, day=15)

In [2]:
from collections import deque, namedtuple
from itertools import chain, count
import math

In [3]:
Combatant = namedtuple('Combatant', 'position team hp attack')

In [4]:
def read_units(text):
    return tuple(chain.from_iterable(
        (Combatant(position=complex(x, y),
                   team='elf' if char == 'E' else 'goblin',
                   hp=200,
                   attack=3)
         for x, char in enumerate(row) if char in ('E', 'G'))
        for y, row in enumerate(text.split('\n'))
    ))

In [5]:
def read_walls(text):
    return set(chain.from_iterable(
        (complex(x, y) for x, char in enumerate(row) if char == '#')
        for y, row in enumerate(text.split('\n'))
    ))

In [6]:
def adjacent(first, second):
    return (first+1 == second or first-1 == second or first+1j == second or first-1j == second)

In [7]:
def free_spaces_next_to_targets(targets, occupied):
    spaces_next_to_targets = chain.from_iterable(
        (target.position+1,
         target.position-1,
         target.position+1j,
         target.position-1j)
        for target in targets
    )
    return set(space for space in spaces_next_to_targets
               if space not in occupied)

In [8]:
def potential_targets(actor, units):
    return set(unit for unit in units if unit.team != actor.team and unit.hp > 0)

In [9]:
def distance_to_spaces(position, occupied):
    compass = (1, -1, 1j, -1j)
    distances = dict()
    consider = deque([(0, position)])

    while consider:
        dist, pos = consider.popleft()
        distances[pos] = dist
        for direction in compass:
            adjacent = pos + direction
            if (adjacent not in occupied) and (adjacent not in distances):
                if not any(c[1] == adjacent for c in consider):
                    consider.append((dist+1, adjacent))

    return distances

In [10]:
def damage(unit, power):
    return Combatant(position=unit.position, team=unit.team, hp=max(0, unit.hp-power), attack=unit.attack)

In [11]:
def moved(unit, position):
    return Combatant(position=position, team=unit.team, hp=unit.hp, attack=unit.attack)

In [12]:
def identify_movement_target(position, enemies, occupied):
    distances = distance_to_spaces(position, occupied)
    reachable_fsntt = {space for space in free_spaces_next_to_targets(enemies, occupied) if space in distances}
    if reachable_fsntt:
        return sorted(reachable_fsntt,
                      key=lambda t: (distances.get(t, math.inf),
                                     t.imag,
                                     t.real))[0]

In [13]:
def next_step_toward_movement_target(position, target, occupied):
    distances = distance_to_spaces(target, occupied)
    adjacent = [loc for loc in (position+1,
                                position-1,
                                position+1j,
                                position-1j)
                if loc not in occupied]
    sort_candidates = lambda c: (distances.get(c, math.inf),
                                 c.imag,
                                 c.real)
    return sorted(adjacent, key=sort_candidates)[0]

In [14]:
def advance(rounds, units, walls):
    sort_units = lambda x: (x.position.imag, x.position.real)

    units = list(sorted(units, key=sort_units))
    victory_before_end = False

    for unit_ix, unit in enumerate(units):
        if unit.hp > 0:
            occupied = walls|{unit.position for id, unit in enumerate(units)
                              if unit.hp > 0 and id != unit_ix}
            adjacent = [loc for loc in (unit.position+1,
                                        unit.position-1,
                                        unit.position+1j,
                                        unit.position-1j)
                        if loc not in walls]

            targets = [other for other in units
                       if other.team != unit.team and other.hp > 0]
            if not targets: # no enemies, so early victory needs recording
                victory_before_end = True
            if (sum(1 for targ in targets if targ.position in adjacent) == 0 and
                adjacent):
                target = identify_movement_target(unit.position,
                                                  targets,
                                                  occupied)
                if target:
                    move_to = next_step_toward_movement_target(unit.position,
                                                               target,
                                                               occupied)
                    unit = moved(unit, move_to)
                    units[unit_ix] = unit

            adjacent = [loc for loc in (unit.position+1,
                                        unit.position-1,
                                        unit.position+1j,
                                        unit.position-1j)
                        if loc not in walls]
            victims = [(id, v) for id, v in enumerate(units)
                       if v.team != unit.team and units[id].hp > 0
                       and v.position in adjacent]
            if victims:
                def sort_victims(victim):
                    id, victim = victim
                    return (victim.hp,
                            victim.position.imag,
                            victim.position.real)
                victid, victim = sorted(victims, key=sort_victims)[0]
                units[victid] = damage(victim, unit.attack)

    rounds = rounds + (0 if victory_before_end else 1)

    return rounds, set(unit for unit in units if unit.hp > 0)

In [15]:
def print_map(round, walls, units):
    map = dict()
    unitdesc = dict()
    for wall in walls:
        map[wall] = '#'
    for unit in sorted(units, key=lambda u: (u.position.imag, u.position.real)):
        teamcode = unit.team[0].upper()
        map[unit.position] = teamcode
        y = unit.position.imag
        unitdesc[y] = unitdesc.get(y, '') + ', {}({})'.format(teamcode, unit.hp)

    max_x = int(max(n.real for n in map.keys()))
    max_y = int(max(n.imag for n in map.keys()))

    header = '{00}\n==\n'.format(round)
    body = '\n'.join(
        (''.join(map.get(complex(x, y), '.') for x in range(1+max_x))
         + ' ' + unitdesc.get(y, ', ')[2:])
        for y in range(1+max_y)
    )

    return header + body + '\n'

In [16]:
def outcome(rounds, units):
    hp = sum(unit.hp for unit in units)
    return (rounds, hp)def main():
    with open(os.path.join(os.path.dirname(__file__), 'input.txt'), 'r') as f:
        input = f.read().strip()
    walls = read_walls(input)
    units = read_units(input)

    # part one:
    rounds, hp = outcome(*battle(walls, units))
    print('part one: {} rounds x {} hp = {}'.format(rounds, hp, rounds*hp))

    # part two:
    rounds, hp = outcome(*find_perfect_battle(walls, units))
    print('part two: {} rounds x {} hp = {}'.format(rounds, hp, rounds*hp))

In [17]:
def battle(walls, units, end_on_first_elf_death=False):
    rounds = 0
    if end_on_first_elf_death:
        elves_needed = sum(1 for unit in units if unit.team == 'elf')
    else:
        elves_needed = 0

    while (len(set(unit.team for unit in units)) > 1 and
           sum(1 for unit in units if unit.team == 'elf') >= elves_needed):
        rounds, units = advance(rounds, units, walls)

    return (rounds, units)

In [18]:
def find_perfect_battle(walls, units):
    elves = {unit for unit in units if unit.team == 'elf'}
    goblins = {unit for unit in units if unit.team == 'goblin'}
    for power in count(4):
        stronger_elves = {Combatant(position=elf.position,
                                    team=elf.team,
                                    hp=elf.hp,
                                    attack=power)
                          for elf in elves}
        rounds, units = battle(walls, stronger_elves|goblins, True)

        if { unit.team for unit in units } == { 'elf' }:
            return rounds, units

In [19]:
walls = read_walls(data)
units = read_units(data)
rd1, hp1 = outcome(*battle(walls, units))
print('Part 1: {} rounds x {} hp = {}'.format(rd1, hp1, rd1*hp1))
rd2, hp2 = outcome(*find_perfect_battle(walls, units))
print('Part 2: {} rounds x {} hp = {}'.format(rd2, hp2, rd2*hp2))

Part 1: 70 rounds x 2713 hp = 189910
Part 2: 59 rounds x 980 hp = 57820
